In [35]:
from cellpaint.utils.args import CellPaintArgs
from cellpaint.utils.helpers_post_feature_extraction import FeaturePreprocessing as FP

import pandas as pd
import numpy as np
import scipy.stats as scstats
import matplotlib.pyplot as plt


from pathlib import WindowsPath

In [23]:
# experiment = "20221116-CP-Fabio-DRC-BM-P02" 
experiment = "20230119-CP-Fabio-QCcelllines-EXP01"
# args = CellPaintArgs(
#     experiment=experiment,
#     main_path="F:\\CellPainting",
#     mode="full",).args
# print(f"{args.experiment}")
main_path= WindowsPath("F:\\CellPainting")
feats_path = main_path/experiment/"Step3_Features"
min_well_cell_count = 100
quantile_range = (2, 98)

In [24]:
features, start_index = FP.load_and_preprocess_features(feats_path, min_well_cell_count)
features = FP.normalize_features(features, start_index, quantile_range)

In [31]:
dmso=features.loc[features["treatment"]=="dmso"]
feat_cols = np.setdiff1d(list(dmso.columns)[start_index:], "Nucleus_Shapes_efc-ratio")
efc_ratio = dmso["Nucleus_Shapes_efc-ratio"]

In [32]:
celllines = np.unique(dmso["cell-line"])
print(celllines)
M = len(celllines)
K = len(feat_cols)

['bt474' 'mcf10a' 'mcf7' 'mdamb231' 'mdamb453' 'skbr3' 't47d' 'u2os']


In [43]:
corr_mat = np.zeros((M, K), dtype=np.float32)
for ii, it in enumerate(celllines):
    cond = dmso["cell-line"] == it
#     corr = np.corrcoef(efc_ratio.loc[cond].to_numpy(), dmso[feat_cols[0]].loc[cond].to_numpy())[0, 1]
#     print(corr)
    corr_mat[ii] = [np.corrcoef(efc_ratio.loc[cond].to_numpy(), dmso[feat_cols[jj]].loc[cond].to_numpy())[0, 1]
                   for jj in range(K)]